<a href="https://colab.research.google.com/github/Preranakh/Advanced-NLP/blob/main/Implement%20word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec: How To Implement word2vec

### Explore Pre-trained Embeddings

Some other options:
- `glove-twitter-{25/50/100/200}`
- `glove-wiki-gigaword-{50/200/300}`
- `word2vec-google-news-300`
- `word2vec-ruscorpora-news-300`

In [ ]:
# Install gensim to get the pretrained model
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load pretrained word vectors using gensim
#EXploring some pre trained embeddings from wikipedia
import gensim.downloader as api

wiki_embeddings = api.load('glove-wiki-gigaword-100')
#each vector should be of length 100

In [ ]:
# Explore the word vector for "king"
wiki_embeddings['king']
#this outputs vector of length 100 for king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

In [ ]:
# Find the words most similar to king based on the trained word vectors
wiki_embeddings.most_similar('king')

[('prince', 0.7682328820228577),
 ('queen', 0.7507690787315369),
 ('son', 0.7020888328552246),
 ('brother', 0.6985775232315063),
 ('monarch', 0.6977890729904175),
 ('throne', 0.6919989585876465),
 ('kingdom', 0.6811409592628479),
 ('father', 0.6802029013633728),
 ('emperor', 0.6712858080863953),
 ('ii', 0.6676074266433716)]

### Train Our Own Model

In [ ]:
# Read in the data and clean up column names
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [ ]:
# Clean data using the built in cleaner in gensim
messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
messages.head()

,label,text,text_clean
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g...","[go, until, jurong, point, crazy, available, only, in, bugis, great, world, la, buffet, cine, th..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,"[free, entry, in, wkly, comp, to, win, fa, cup, final, tkts, st, may, text, fa, to, to, receive,..."
3,ham,U dun say so early hor... U c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives around here though","[nah, don, think, he, goes, to, usf, he, lives, around, here, though]"


In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

In [ ]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   # this is size of vector
                                   window=5,
                                   #window defines the number of words before and after the context of the word
                                   min_count=2)

In [ ]:
# Explore the word vector for "king" base on our trained model
w2v_model.wv['king']

array([-3.0207470e-02,  4.3379463e-02,  5.4586465e-03,  2.2445803e-03,
        1.9619584e-02, -9.8471828e-02,  2.8901469e-02,  1.0937159e-01,
       -3.9308980e-02, -2.5372971e-02, -2.0290151e-02, -9.2733786e-02,
       -7.1460688e-03,  2.2716038e-02,  2.5377374e-02, -2.0540904e-02,
        2.4363901e-02, -6.5424383e-02, -1.3421078e-02, -1.0664750e-01,
        4.5222931e-02,  2.1314444e-02,  3.8848970e-02, -3.8184173e-02,
       -3.0569134e-02,  1.0484788e-02, -4.8772175e-02, -1.5628947e-02,
       -5.2985173e-02,  1.2321942e-02,  6.5337561e-02, -7.7670394e-03,
        3.0335149e-02, -4.8792299e-02, -3.4876566e-02,  8.1594683e-02,
       -1.2421133e-03, -4.1978274e-02, -3.9664008e-02, -9.8295242e-02,
        1.4416549e-02, -5.7889096e-02, -3.7245233e-02,  7.1273874e-03,
        2.7308369e-02, -3.6724251e-02, -5.5061676e-02, -2.1752965e-02,
        3.0644644e-02,  2.6593672e-02,  1.2545275e-02, -6.4166859e-02,
       -1.1797749e-02, -4.2855144e-03, -1.9706886e-02,  3.9236531e-02,
      

In [ ]:
# Find the most similar words to "king" based on word vectors from our trained model
w2v_model.wv.most_similar('king')

[('lets', 0.992987871170044),
 ('felt', 0.9929654002189636),
 ('pobox', 0.9929404854774475),
 ('ah', 0.9929193258285522),
 ('tomo', 0.9928525686264038),
 ('without', 0.9928063750267029),
 ('gud', 0.9927749633789062),
 ('sent', 0.9927712678909302),
 ('which', 0.9927339553833008),
 ('some', 0.9927164316177368)]

In [ ]:
#when we do this with wikipedia, the results made a lot more sense, we saw prince, son etc, but now these similar words does not make any sense
# so we can say that wikipedia wods embeddings are better